<a href="https://colab.research.google.com/github/Kkhokho/Movie_Recommendation_System/blob/main/Recommendation_system_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [8]:

# IMDb movies.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('IMDb movies.csv')
nRow, nCol = df1.shape

<ipython-input-8-06c69bb97719>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('IMDb movies.csv')


In [6]:
import math

In [7]:
! pip install faiss-gpu ##Installing GPU version of faiss


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00


In [8]:
! pip install sentence_transformers ## For textual similarity, using pretrained models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d8fee1df9a3c688413853e7d8dd16fdf1988f36c9ce09e8cee467ae25e7f3fb9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [12]:
df1

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,tt9908390,Le lion,Le lion,2020,2020-01-29,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,...,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,5.3,398,NaN,NaN,$ 3507171,NaN,NaN,4.0
85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,2020,2020-02-13,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,...,"Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,7.7,724,NaN,NaN,$ 7299062,NaN,6.0,4.0
85852,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",NaN,7.9,265,NaN,NaN,NaN,NaN,NaN,NaN
85853,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,2019-03-15,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,...,"Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",NaN,6.4,194,NaN,NaN,$ 2833,NaN,NaN,NaN


In [2]:
imdb_movies=pd.read_csv("/content/IMDb movies.csv")
imdb_movies=imdb_movies[pd.notnull(imdb_movies['description'])]
imdb_movies=imdb_movies.reset_index(drop=True)

<ipython-input-2-04ad4d1a733c>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_movies=pd.read_csv("/content/IMDb movies.csv")


In [3]:
imdb_movies['id']=imdb_movies.index

In [4]:
sentences=imdb_movies['description'].tolist()
print("Number of Sentences in Movie Description ",len(sentences))

Number of Sentences in Movie Description  83740


In [9]:
import faiss
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-distilroberta-base-v1',device="cuda")

In [10]:
embeddings=model.encode(sentences)
faiss.normalize_L2(embeddings)

In [11]:
embeddings.shape

(83740, 768)

In [12]:
dim=768
ncentroids=50 ## This is a hyperparameter, and indicates number of clusters to be split into
m=16 ## This is also a hyper parameter
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ (quantiser, dim,ncentroids, m , 8)
index.train(embeddings) ## This step, will do the clustering and create the clusters
print(index.is_trained)
faiss.write_index(index, "trained.index")

True


In [13]:
ids=imdb_movies['id'].tolist()
ids=np.array(ids)
index.add_with_ids(embeddings,ids)
print(index.ntotal)

83740


In [14]:
faiss.write_index(index,"block.index")

In [15]:
def searchFAISSIndex(data,id_col_name,query,index,nprobe,model,topk=20):
    ## Convert the query into embeddings
    query_embedding=model.encode([query])[0]
    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    faiss.normalize_L2(query_embedding)


    index.nprobe=nprobe

    D,I=index.search(query_embedding,topk)
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score]
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [16]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

In [17]:
query="A seventeen-year-old aristocrat falls in love with a kind but poor artist"
search_result=searchFAISSIndex(imdb_movies,"id",query,index,nprobe=10,model=model,topk=20)
search_result=search_result[['id','description','title','cosine_sim','L2_score']]

In [18]:
search_result

,id,description,title,cosine_sim,L2_score
12,30703,A seventeen-year-old aristocrat falls in love ...,Titanic,0.834949,0.574545
6,4689,An English lady falls madly in love with a Fre...,L'avventura viene dal mare,0.792439,0.644299
15,45478,A film director falls in love with the daughte...,Il regista di matrimoni,0.787769,0.651507
1,682,A young impoverished aristocrat falls in love ...,Sinfonia nuziale,0.783159,0.658545
0,674,"In Czarist Russia, a peasant officer, resented...",Nella tempesta,0.782185,0.660023
17,59293,The only daughter of a wealthy businessman fal...,Love U... Mr. Kalakaar!,0.782102,0.660149
7,5393,A beautiful young pianist falls for her mentor.,Gli anni più belli,0.772382,0.674712
14,41846,Suburban priest falls in love with woman in wh...,Miffo,0.767952,0.681245
8,5643,A good natured soldier with a killer right han...,Soldaten og Jenny,0.766140,0.683901
16,58571,A popular singer with two identities falls in ...,Kajraare,0.765014,0.685545


In [19]:
query="A intelligent boy plays football"
search_result=searchFAISSIndex(imdb_movies,"id",query,index,nprobe=10,model=model,topk=20)
search_result=search_result[['id','description','title','cosine_sim','L2_score']]

In [20]:
search_result

,id,description,title,cosine_sim,L2_score
5,30764,A waterboy for a college football team discove...,Waterboy,0.667577,0.815381
6,31308,A young boy with stunted growth is convinced t...,Simon Birch,0.583029,0.913204
14,59679,"Ivan, an eleven-year-old boy, plays against th...",El sueño de Iván,0.580007,0.916507
13,58316,A drama centered around a promising football r...,All Things Fall Apart,0.576196,0.920657
17,72877,A young man is badly injured in a football sta...,Hame Chiz Baraye Foroosh,0.570356,0.926977
0,7001,The greatest player in Ridgefield College hist...,Quel fenomeno di mio figlio,0.569434,0.927972
19,81931,"An ordinary working class boy, like all his pe...",Lev Yashin. Vratar moey mechty,0.562063,0.935882
15,69466,Recep Ivedik coaches the children football tea...,Recep Ivedik 4,0.557117,0.941152
16,69675,"The love of two schoolboys, Golf and Bank. Gol...",Phi chai,0.552909,0.945612
9,39846,A playboy mends his reckless ways and learns t...,Hum Ho Gaye Aap Ke,0.550460,0.948198
